### 验证码识别
- 相关的线上打码平台识别
    - 打码兔
    - 云打码
    - 超级鹰：http://www.chaojiying.com/about.html
        - 1. 注册，登录
        - 2. 创建一个软件：软件ID->生成一个软件ID：	916676
        - 3. 下载示例代码


In [2]:
#!/usr/bin/env python
# coding:utf-8

import requests
from hashlib import md5

class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()


if __name__ == '__main__':
    chaojiying = Chaojiying_Client('15588545521', 'lovedc123', '916676')	#用户中心>>软件ID 生成一个替换 96001
    im = open('a.jpg', 'rb').read()													#本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
    print(chaojiying.PostPic(im, 1004).get("pic_str"))												#1902 验证码类型  官方网站>>价格体系 3.4+版 print 后要加()



7261


- 将古诗文网中验证码图片进行识别：https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx

In [18]:
def discernVertiCode(imgPath,codeType):
    chaojiying = Chaojiying_Client('15588545521', 'lovedc123', '916676') #用户中心>>软件ID 生成一个替换 96001
    im = open(imgPath, 'rb').read() #本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
    return chaojiying.PostPic(im, codeType).get("pic_str") #1902 验证码类型  官方网站>>价格体系 3.4+版 print 后要加()

In [19]:
import requests
from lxml import etree
url = "https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx"
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11"
}
res = requests.get(url,headers=headers)
res.encoding = "utf8"
pageText = res.text
tree = etree.HTML(pageText)
imgSrc = tree.xpath("//form/div[4]/div[4]/img/@src")[0]
urlImg = "https://so.gushiwen.org"+imgSrc
img = requests.get(urlImg,headers=headers).content
with open("./vertiCode.png","wb") as f:
    f.write(img)
print("爬取完毕")
codeStr = discernVertiCode("./vertiCode.png",1004)
print(codeStr)


爬取完毕
aewo


- 为什么在爬虫中需要实现模拟登录？
    - 有的数据必须经过登录后，才可以显示
- 涉及到的反爬
    - 验证码
    - 动态请求参数：每次请求对应的请求参数都是动态变化
        - 动态捕获：通常情况下，动态的请求参数都会被隐藏在前台页面的源码
    - Cookie

In [25]:
import requests
from lxml import etree

url = "https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx"
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11"
}
res = requests.get(url,headers=headers)
res.encoding = "utf8"
pageText = res.text
tree = etree.HTML(pageText)
imgSrc = tree.xpath("//form/div[4]/div[4]/img/@src")[0]
# 动态捕获动态请求参数 
viewstate = tree.xpath('//*[@id="__VIEWSTATE"]/@value')[0]
viewstategenerator = tree.xpath('//*[@id="__VIEWSTATEGENERATOR"]/@value')[0]
urlImg = "https://so.gushiwen.org"+imgSrc
sess = requests.Session()
img = sess.get(urlImg,headers=headers).content
with open("./vertiCode.png","wb") as f:
    f.write(img)
codeStr = discernVertiCode("./vertiCode.png",1004)
loginUrl = r"https://so.gushiwen.org/user/login.aspx?from=http%3a%2f%2fso.gushiwen.org%2fuser%2fcollect.aspx"
data = {
    "__VIEWSTATE": viewstate,
    "__VIEWSTATEGENERATOR": viewstategenerator,
    "from": "http://so.gushiwen.org/user/collect.aspx",
    "email": "15588545521",
    "pwd": "lovedc123",
    "code": codeStr,
    "denglu": "登录"
}
loginText = sess.post(loginUrl,data,headers=headers).text
with open("./gushiwen.html","w",encoding="utf8") as f:
    f.write(loginText)